In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.01):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.learning_rate = learning_rate

        self.weights_input_hidden = np.random.rand(self.input_size, self.hidden_size)
        self.bias_hidden = np.random.rand(self.hidden_size)
        self.weights_hidden_output = np.random.rand(self.hidden_size, self.output_size)
        self.bias_output = np.random.rand(self.output_size)

    def forward(self, inputs):
        self.hidden_layer_input = np.dot(inputs, self.weights_input_hidden) + self.bias_hidden
        self.hidden_layer_output = sigmoid(self.hidden_layer_input)
        self.output_layer_input = np.dot(self.hidden_layer_output, self.weights_hidden_output) + self.bias_output
        self.output_layer_output = sigmoid(self.output_layer_input)
        return self.output_layer_output

    def backward(self, inputs, expected_output, actual_output):
        output_error = expected_output - actual_output
        output_delta = output_error * sigmoid_derivative(actual_output)

        hidden_error = output_delta.dot(self.weights_hidden_output.T)
        hidden_delta = hidden_error * sigmoid_derivative(self.hidden_layer_output)

        self.weights_hidden_output += self.hidden_layer_output.T.dot(output_delta) * self.learning_rate
        self.bias_output += np.sum(output_delta, axis=0) * self.learning_rate
        self.weights_input_hidden += inputs.T.dot(hidden_delta) * self.learning_rate
        self.bias_hidden += np.sum(hidden_delta, axis=0) * self.learning_rate

    def train(self, inputs, expected_output, epochs):
        for epoch in range(epochs):
            actual_output = self.forward(inputs)
            self.backward(inputs, expected_output, actual_output)
            if epoch % 100 == 0:
                loss = np.mean((expected_output - actual_output) ** 2)
                print(f"Epoch {epoch}, Loss: {loss}")

def create_windows(data, window_size):
    X = []
    y = []
    for i in range(len(data) - window_size):
        X.append(data[i:i + window_size, :-1])  
        y.append(data[i + window_size, -1])     
    return np.array(X), np.array(y)

file_path = 'LIZA.dat'
data = pd.read_csv(file_path, delim_whitespace=True, skiprows=1)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
print("Усі рядки зчитаних даних:")
print(data)
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')


X = data.drop('IPC', axis=1).values  
y = data['IPC'].values  

scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(X)
y_normalized = scaler.fit_transform(y.reshape(-1, 1)).flatten()

data_normalized = np.hstack((X_normalized, y_normalized.reshape(-1, 1)))
window_size = 5 
X, y = create_windows(data_normalized, window_size)

X = X.reshape(X.shape[0], -1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

nn = NeuralNetwork(input_size=X_train.shape[1], hidden_size=8, output_size=1, learning_rate=0.1)
nn.train(X_train, y_train.reshape(-1, 1), epochs=1000)

predictions = [nn.forward(x) for x in X_test]

y_test_rescaled = scaler.inverse_transform([[0] * X.shape[1] + [val] for val in y_test])[:, -1]
predictions_rescaled = scaler.inverse_transform([[0] * X.shape[1] + [pred[0]] for pred in predictions])[:, -1]

overall_mse_rescaled = np.mean((y_test_rescaled - predictions_rescaled) ** 2)
print(f"\nЗагальне MSE у оригінальному масштабі: {overall_mse_rescaled:.6f}")

results = []

print("\nПрогноз для кожного вікна:")
for i in range(len(X_test)):
    window_input = X_test[i].reshape(1, -1) 
    predicted_value = nn.forward(window_input).flatten()  
    
    real_value = scaler.inverse_transform([[0] * X.shape[1] + [y_test[i]]])[0, -1]
    
    predicted_value_rescaled = scaler.inverse_transform([[0] * X.shape[1] + [predicted_value[0]]])[0, -1]
    
    mse = (real_value - predicted_value_rescaled) ** 2
    
    results.append({
        "Window": i + 1,
        "Real Value": real_value,
        "Predicted Value": predicted_value_rescaled,
        "MSE": mse
    })

df_results = pd.DataFrame(results)

average_mse = df_results['MSE'].mean()
print(f"\nСереднє MSE для тестових вікон: {average_mse:.6f}")

print("\nРезультати порівняння:")
print(df_results)

df_results.to_csv('window_results.csv', index=False)

